<div>
    <img style="float:right;" src="images/smi-logo.png"/>
    <div style="float:left;color:#58288C;"><h1>Introduction to Python for Data Science</h1></div>
</div>

---
# Notebook 3: Pandas
This notebook introduces the `pandas` package as a convenient toolset to work with tabular data.

## Contents

[1. Importing data from APIs](#chapter1)  
[2. Introduction to DataFrames](#chapter2)  
[3. Simple data visualization](#chapter3)  
[4. INSIDER Task](#chapter4)  
---

# 1. Importing data from APIs <a id="chapter1"/>

We'll start this session by using [REST-APIs](https://en.wikipedia.org/wiki/Representational_state_transfer) to retrieve some data. In short, when using a REST API, we use the same methods as a browser does, when retrieving a webpage. But instead of an HTML description of a webpage, we retrieve the data.

For practising , [this](https://github.com/public-apis/public-apis) is a list of publicly available APIs. In this notebook we are going to use `corona-api.com` that provides recent COVID infection data from the WHO and Johns Hopkins university in a compact data format.

To get a first impression, point your browser to http://corona-api.com/timeline. The displayed data is a mixed data structure, some sections correspond to Python lists, others to dictionaries.

Let's retrieve this data, step by step...

In [ ]:
import requests    # package to send http queries to the API

link = "http://corona-api.com/timeline"   # URL to query, you can try http://corona-api.com/countries/DE instead or replace DE with another country code

res = requests.get("https://corona-api.com/timeline")   # send a get request to that url, store the response in variable "res"
raw_data = res.json()  # this now contains the data, uncomment this line and execute the cell to check

---
## <span style="color:#FF5D02;">Assignment: Analyze data structure </span>

The retrieved raw data should look like this:

```
{'data': [  
  {'updated_at': '...',
   'date': '2022-09-13',
   'deaths': 6467297,
   'confirmed': 604938009,
   'recovered': 0,
   'new_confirmed': 508345,
   'new_recovered': 0,
   'new_deaths': 1446,
   'active': 598470712},
  {'updated_at': '...',
   'date': '2022-09-12',
   'deaths': 6465852,
   'confirmed': 604429805,
   'recovered': 0,
   'new_confirmed': 313811,
   'new_recovered': 0,
   'new_deaths': 740,
   'active': 597963953},
   ...
  ]
```

Please take a minute to describe for yourself what datastructures you recognize! Is it a list of dictionaries? A dictionary with keys that contain lists? A dictionary of dictionaries?


Try to access some of the data fields with the syntax you've learned previously to access dictionaries and lists:

In [ ]:
raw_data[...][...]

Expand the following two cells to see the solution!

In [ ]:
raw_data["data"][0] # this returns the most recent record

In [ ]:
raw_data["data"][0]["confirmed"] # this returns the number of confirmed cases from the most recent record

---

# 2. Introduction to DataFrames<a id="chapter2"/>
You probably noticed that working purely with lists of dictionaries and such is not 
Pandas are the central tool for reading and manipulating data in Python. For our purposes, the `DataFrames` data structure is the most important:

> **DataFrame** is a 2-dimensional labeled data structure with columns of potentially different types.  
> You can think of it like a spreadsheet or SQL table [...]. It is generally the most commonly used pandas object.  
> [(Source)](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html)

Dataframes contain rows and columns and distinguish between regular data and indices - columns that contain an unique identifier for each row.
I.e. our just imported covid dataset would look like this:

<img src="images/dataframe.png"/>  

The full documentation can be found [here](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html). 

Let's explore step by step, how dataframes make life easier when exploring data.

In [ ]:
import pandas
import requests

raw_data = requests.get("https://corona-api.com/timeline").json()  # fetch data
df = pandas.DataFrame(raw_data["data"])     # raw_data["data"] contains the relevant datatable as a list of dictionaries [{},{},{},{},....], see above

In [ ]:
# let's have a look at our brand new dataframe...
df

In [ ]:
df.head(5)   # shows the first n rows

In [ ]:
df.sample(5) # show random n rows

In [ ]:
df.count()   # shows the number of valid entries per column

## 2.1. Preparing the dataset

Before working with the data, we usually want to remove/rename some columns, sort the data, apply filters or partition the data.
In this chapter we'll briefly walk the some commonly used functions to prepare datasets.

> **Important**: All edits to the Dataframe create a copy with the changes, if you don't explicitly force the function to apply the changes directly ("inplace"). If you don't force inplace editing, the original DataFrame remains unchanged. So you usually have two options to apply changes:  
>
> `df = df.change_something(...)             # assign the copy with the change to the original variable`  
> `df.change_something(..., inplace=True)    # apply the change to the original dataframe`

In [ ]:
# Let's prepare the dataframe...

# First delete / rename some columns
df = df.drop(["updated_at", "deaths", "confirmed", "new_recovered", "recovered"], axis="columns")

df = df.rename(columns = {                      # pass a dictionary of "oldname": "newname" pairs to rename columns
    "new_confirmed": "new_cases", 
})

df = df.sort_values("date", ascending=True)     # sort data ascending

df = df.set_index("date")                       # set date column as unique identifier for records (index)

In [ ]:
df.head() # check result

## 2.2. Selections and filtering

Dataframes generally accept filters/selections in the format `[row_filter, column_filter]`. The expressions and inner workings can be quite different, we look at some of the most helpful ways.

### Select by True/False vector

In [ ]:
# by passing a vector of True and False as row_filter, we toggle which rows we want to keep
# a bool condition like the following generates such a structure, in this case with the date as index ... try it!

df.new_cases > 30000

In [ ]:
# let's pass this as row_filter
peaks = df[df.new_cases > 30000]                                # select all days with > 30.000 cases
peaks.sort_values("new_cases", ascending=False)                 # show dataframe, sorted by "worst days" first

In [ ]:
df[(df.new_cases > 30000) & (df.new_deaths < 600)]              # Use bool algebra operators & ("and") and | ("or") to combine filters

In [ ]:
df[(df.new_cases > 30000) | (df.new_deaths < 600)]              # Important: don't forget the brackets

### Select by naming relevant rows, columns

In [ ]:
# Select using function .loc[list of row_indexes, list of column names]:

df.loc["2020-08-01","cases"]   # single day, single column

In [ ]:
df.loc["2020-09-01":"2020-09-07", "deaths":"active"]  # ranges of days, range of columns

In [ ]:
df.loc["2020-09-01":"2020-09-07", : ]  # ranges of days, all columns (full range

In [ ]:
df.loc[["2020-02-01","2020-03-01"], ["deaths", "active"]]  # subsets via lists of days (index) and columns

---
## <span style="color:#FF5D02;">Assignment: Data selection</span>

Generate a new dataframe that only contains new_cases for January 2021!

---

## 2.3. Calculations and simple statistics

In [ ]:
# Do calculations with columns similar to single variables

df["death_rate"] = df.new_deaths / df.active

In [ ]:
# Calculate common descriptive statistics for numeric columns of the whole dataframe
df.describe()

In [ ]:
# ... or for a single column
df.new_cases.describe()   # note the scientific notation ("e notation") in the result, if unknown, check here: https://en.wikipedia.org/wiki/Scientific_notation

In [ ]:
# or just a specific metric :-)

print("New Cases")
print("Mean: ", df.new_cases.mean())
print("Median: ", df.new_cases.median())
print("Maximum: ", df.new_cases.max())
print("20% quantile: ", df.new_cases.quantile(0.2))
print("80% quantile: ", df.new_cases.quantile(0.8))

---
## <span style="color:#FF5D02;">Assignment: Compare descriptive statistics</span>

Calculate the average of new cases of Jan 2021 and Feb 2021! Which one is bigger?

Try it first, if you need hints, expand the following cell:

> **Hints**
> - Use the .loc[] function to select the months (see above examples for index ranges)
> - Use the .mean() function to calculate the mean

---
# 3. Simple data visualization<a id="chapter3"/>

There are numerous data visualization packages available for Python (e.g. matplotlib, seaborn). The [Python Graph Gallery](https://www.python-graph-gallery.com) gives a lot of examples with code snippets.

Pandas include a `.plot()` function that automatically calls the respective functionality from a visualization package (matplotlib, by default).

This sections shows a lot of examples to get you started.

In [ ]:
last_quarter = df.new_cases[-90:-1]
last_quarter.plot()  # plot the new cases for the last 90 days 

# (if you're unsure about the syntax, recheck the Python basics notebook or google python negative indexing for lists or dataframes 

In [ ]:
# let's make it look a little nicer, for a list of all parameters check https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html

last_quarter.plot(kind="area", figsize=(18,5), 
                  color="lightblue", legend=True,
                   title="New infections during the last quarter",
                   ylabel="Number of cases")

In [ ]:
# let's check the distribution of daily case count for January 2021 with a boxplot
last_quarter.plot(kind="box")

In [ ]:
# let's do a histogram to check the overall distribution
last_quarter.plot(kind="hist", edgecolor="white") 

In [ ]:
# For scatterplots (plot x vs y values as dots) we need a little different syntax and come back to the full dataframe, containing all columns:
df[-90:-1].plot(kind="scatter", x="new_cases", y="new_deaths", color="blue", title="Todesfälle vs. Neuinfektionen pro Tag")

To show multiple data series in a single plot, just put the statements in the same notebook cell:

In [ ]:
df[-90:-1].new_cases.plot  (kind="hist", figsize=(16,5), alpha=0.5, color="blue", legend=True, 
                            label="last 90 days", title="Number of days with x new infections, quarterly comparison") 
df[-180:-91].new_cases.plot(kind="hist", figsize=(16,5), alpha=0.2, color="green", legend=True, 
                            label="previous 90 days") 

# 4. INSIDER Task<a id="chapter4"/>

### Your task as group of 2-3 ppl:
- Briefly check out https://teleport.org to understand what data this assignment is about
- Pick 3-5 big cities that you're interested in benchmarking
- Use the [teleport API](https://developers.teleport.org/api/) to compile a single table that lists all life quality scores for the respective urban areas. The resulting table should look like this
   Name | City1 | City2 | City3 | ...
   :-----------|:--:|:--:|:--:|:--:
   Housing | 7.2 | 6.9 | ... | ...
   Cost of Living | 5.7 | 9.1 | ... | ...
   ... | ... | ... | ... | ...
- Compute the total score of the urban areas by averaging all individual scores of each area
- Round all scores to one decimal digit and add the mean scores to your result dataframe
- Generate a horizontal bar chart to visualize your findings
- Make sure, all key steps of your code have concise english comments
- Submit your commented code + the mean scores of your cities as solution to the INSIDER task

> *Remarks regarding scoring of this task*: Task is considered basically completed if your code produces a table of correct scores for each urban area. The task is done well if your code generates a single table with all scores, can display the demanded bar chart and compute the means successfully. The task is done perfect if you round all values, append the mean values to the table and this way include the mean scores in your bar chart.

### Hints

#### General

- Develop your code in multiple cells so you can check the output of every step
- For the final solution you will need a loop to fetch data for all the cities, but don't start this way. First get the task working for a single city, then carefully introduce more complexity to your code.
- You have seen all required operations in the notebooks you worked through for this task; feel free to use this as copy/paste reference. Where unsure use google to find more examples of a certain operation.
- Getting the quality scores from teleport API requires going through these steps: 
  * look up the city, get the link to the city data
  * use the city data link to get the link to the urban area of the city
  * use the urban area data link to get the link to the scores
  * retrieve the scores
- You can save your work by simply copy/pasting your code to an empty document on your computer or downloading this notebook (File -> Download). After re-opening the Jupyter environment, you can re-upload the notebook by drag/dropping the downloaded notebook to the file explorer on the left hand side.

#### Working with API results

- Before writing the code, you might want to walk through the sequence of queries in a browser. Start with the city search and copy/paste the next required link. They look like this:
   * Link to city search: `http://api.teleport.org/api/cities/?search=CityName`
   * Link to city data: `https://api.teleport.org/api/cities/geonameid:???????/`
   * Link to urban area: `https://api.teleport.org/api/urban_areas/slug:???????/`
   * Link to scores: `https://api.teleport.org/api/urban_areas/slug:??????/scores/`
- Getting the required bits of information from the retrieved raw (JSON) data is a little cumbersome. The cell below contains an example query that demonstrates how to dig layer by layer into the first API response. If this feels confusing, revisit notebook #2 and recheck the sections about lists and dictionaries, especially the person example.

#### Preparing your dataframe

- Remember: operations like drop, rename, etc. create and return a copy of the dataframe. Use inplace editing like `df.rename(..., inplace=True)` or assign the changed dataset to the original name like in `df = df.rename(...)` to make your changes persistent
- To approach the problem stepwise, create and prepare separate dataframes for each city in your loop and join the dataframes at the end outside the loop. 
- In might be useful to drop the color column and rename column "score_out_of_10" to the city name in dataframe preparation.
- Don't forget to set the index. 
- There are at least two options to join multiple dataframes:
   * Use the `.loc[]` function to piece your result dataframe together, like `df.loc[:,"columnX"] = ...` to add a column
   * Check the documentation of `pandas.concat()` function online and use it to generate your result

## Some things to get you started...
You can start working directly in the cell below or create a new notebook with File->New->Notebook and copy the following code.

In [ ]:
import requests
import pandas

# Entry link to teleport API
url_citysearch = "http://api.teleport.org/api/cities/?search="   # append the name of the city you want to search for at the end of this string

# Cities to analyze
cities = ["city1", "city2", "city3", ...]
scores = []                                  # empty list, your loop can add the results here later

# Starting point / example: Lookup of exemplary city

result = requests.get(url_citysearch + "Berlin")
link_to_city_data = result.json()["_embedded"]["city:search-results"][0]["_links"]["city:item"]["href"]
print(link_to_city_data)

# ...